In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0,1,2,3'
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
from niwidgets import NiftiWidget
import tensorflow as tf
from tensorflow.keras.preprocessing.image import * 
from tensorflow import keras
from config import *
import math
import cv2
import random
from skimage.transform import rescale, resize, downscale_local_mean,rotate
import numpy as np
from skimage.util import montage, crop
from medio import read_img, save_img
from tensorflow.python.client import device_lib
from tensorflow.keras.metrics import MeanIoU,BinaryCrossentropy,Accuracy,Precision,Recall
from tensorflow.keras.models import load_model
from skimage.transform import rescale, resize, downscale_local_mean,rotate
from skimage import data, color
from scipy import ndimage
from scipy import misc
from numpy import fliplr



/home/jain0000/miniconda3/envs/segment1/lib/python3.7/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [2]:
def resize_data(image,shape=(40,40,80)):

    resize_image = resize(rescale(image, 1,anti_aliasing=False), shape, anti_aliasing=True) #inputs.get_fdata()[5:45, 5: 45, 10:90]   
    return (resize_image)


def dice_coeff(y_true, y_pred):
    smooth = 1.
    # Flatten
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)
    return score

def print_metrics(mask,pred_mask,model,niiImage ):
    
    m = MeanIoU(num_classes=2)
    m.update_state(mask.flatten(), pred_mask.flatten())


    m2 = BinaryCrossentropy()
    m2.update_state(mask.flatten(), pred_mask.flatten())

 #   m3 = dice_coeff(resize_mask,pred_mask) 

    m4 = Accuracy()
    m4.update_state(mask.flatten(), pred_mask.flatten())

    m5 = Precision()
    m5.update_state(mask.flatten(), pred_mask.flatten())

    m6 = Recall()
    m6.update_state(mask.flatten(), pred_mask.flatten())

    
    print("meanIOU",m.result().numpy(),"Accuracy",m4.result().numpy(),
         "Precision", m5.result().numpy(),"Recall",m6.result().numpy()," Binary cross entropy",m2.result().numpy())
    
     # create logger
        
    logging.debug('Model: %s ,Tooth sample : %s,meanIOU : %s , Accuracy : %s , Precision : %s, Recall: %s, Binary cross entropy : %s',model, niiImage ,m.result().numpy(),m4.result().numpy(),
         m5.result().numpy(),m6.result().numpy() ,m2.result().numpy())
    # define a Handler which writes INFO messages or higher to the sys.stderr
    return None

import logging
from skimage.measure import marching_cubes_lewiner
import meshplot as mp

def test():
    for m in os.listdir('model/UNET3D/UNET3D-ToothSegmentation-MSE1/'):    
        print(m)
        model = load_model('model/UNET3D/UNET3D-ToothSegmentation-MSE1/' + m)

        for niiImage in os.listdir(data_dir_test_image):

                inputs = nib.load( os.path.join(data_dir_test_image, niiImage)).get_fdata()   
                mask = nib.load( os.path.join(data_dir_test_mask, niiImage)).get_fdata() 

                print(inputs.shape)
                logging.basicConfig(level=logging.DEBUG,
                        format='%(asctime)s %(levelname)s %(message)s',
                        filename='myapp.log',
                        filemode='a')
              
               
                console = logging.StreamHandler()
                console.setLevel(logging.DEBUG)
                # set a format which is simpler for console use
                formatter = logging.Formatter('%(name)-12s: %(levelname)-8s %(message)s')
                # tell the handler to use this format
                console.setFormatter(formatter)
                # add the handler to the root logger
                logging.getLogger('').addHandler(console)


                resize_image = resize(inputs, (40, 40, 80), anti_aliasing=True)
                resize_mask = resize(mask, (40, 40, 80), anti_aliasing=True)

                img_np = np.rollaxis(np.array(np.array([np.array([np.array(resize_image)]) ])),0,5)

                pred = model.predict(img_np) > 0.5
                pred_mask  = resize_data(np.transpose(np.transpose(pred[0])[0]),mask.shape) 


                img = nib.Nifti1Image(pred_mask, np.eye(4))
                if os.path.isdir('segmented-volume/'+m+'/'):
                    nib.save(img, os.path.join('segmented-volume/'+m+'/', niiImage))  
                else:
                    directory = os.mkdir('segmented-volume/'+m+'/')

                x = print_metrics(mask,pred_mask,str(m),niiImage)


                vertices,faces,_,_ = marching_cubes_lewiner(pred_mask)

                mp.plot(vertices, faces, return_plot=False)

    return (img_np,pred_mask)

test()

UNET3D-ToothSegmentation-scale_0.5-rotation_None.h5
(65, 53, 108)


The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
The default multichannel argument (None) is deprecated.  Please specify either True or False explicitly.  multichannel will default to False starting with release 0.16.
root        : DEBUG    Model: UNET3D-ToothSegmentation-scale_0.5-rotation_None.h5 ,Tooth sample : 29.6.67.nii,meanIOU : 0.2874687 , Accuracy : 0.3442348 , Precision : 0.17667076, Recall: 0.9386699, Binary cross entropy : 8.22402


meanIOU 0.2874687 Accuracy 0.3442348 Precision 0.17667076 Recall 0.9386699  Binary cross entropy 8.22402


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(32.0, 26.…

(50, 50, 100)


...

(82, 60, 94)


root        : DEBUG    Model: UNET3D-ToothSegmentation-scale_None-rotation_10.h5 ,Tooth sample : 29.11.92.nii,meanIOU : 0.8850363 , Accuracy : 0.9279947 , Precision : 0.8398761, Recall: 0.94758785, Binary cross entropy : 0.13259809
root        : DEBUG    Model: UNET3D-ToothSegmentation-scale_None-rotation_10.h5 ,Tooth sample : 29.11.92.nii,meanIOU : 0.8850363 , Accuracy : 0.9279947 , Precision : 0.8398761, Recall: 0.94758785, Binary cross entropy : 0.13259809
root        : DEBUG    Model: UNET3D-ToothSegmentation-scale_None-rotation_10.h5 ,Tooth sample : 29.11.92.nii,meanIOU : 0.8850363 , Accuracy : 0.9279947 , Precision : 0.8398761, Recall: 0.94758785, Binary cross entropy : 0.13259809
root        : DEBUG    Model: UNET3D-ToothSegmentation-scale_None-rotation_10.h5 ,Tooth sample : 29.11.92.nii,meanIOU : 0.8850363 , Accuracy : 0.9279947 , Precision : 0.8398761, Recall: 0.94758785, Binary cross entropy : 0.13259809
root        : DEBUG    Model: UNET3D-ToothSegmentation-scale_None-rotati

meanIOU 0.8850363 Accuracy 0.9279947 Precision 0.8398761 Recall 0.94758785  Binary cross entropy 0.13259809


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(44.600000…

(array([[[[[1096.56418659],
           [1098.7562069 ],
           [1045.38949336],
           ...,
           [ 284.14585992],
           [ 287.97049752],
           [ 320.26375292]],
 
          [[1045.46946743],
           [1043.08011014],
           [1031.06808873],
           ...,
           [ 279.19730383],
           [ 320.9938076 ],
           [ 341.19081781]],
 
          [[ 965.886849  ],
           [1020.061967  ],
           [1039.92704598],
           ...,
           [ 348.16808478],
           [ 363.74791356],
           [ 354.88273159]],
 
          ...,
 
          [[  87.27733286],
           [ 136.50564149],
           [ 160.84165302],
           ...,
           [ 242.03409083],
           [ 263.7306867 ],
           [ 275.52095467]],
 
          [[ 113.3438256 ],
           [ 127.05752877],
           [ 153.42869741],
           ...,
           [ 237.51710148],
           [ 262.18002826],
           [ 292.58046348]],
 
          [[ 137.20148458],
           [ 123.008

In [3]:
import pandas as pd


df = pd.read_table('myapp.log',',|:|-|_|.h5',header=None  )  
df=df.drop(df.columns[[0,1,2,3,4,5,6,7,8,10,12,13,15,17,19,21,23]],axis=1)
df
df.columns=['Scale','Rotation', 'Sample', 'meanIOU','Accuracy', 'Precision','Recall','Binary cross entropy']
df
gk = df.groupby(['Sample','Scale','Rotation'],axis=0).mean()
gk

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


meanIOU  Accuracy  Precision    Recall  \
Sample        Scale Rotation                                            
 14.8.93.nii  0.25  10        0.695390  0.863644   0.799946  0.631646   
                    None      0.716429  0.864595   0.780896  0.676934   
              0.5   10        0.703036  0.860019   0.769226  0.652287   
                    None      0.695139  0.855342   0.761618  0.658707   
              2.0   10        0.722769  0.863099   0.772593  0.698822   
                    None      0.698896  0.855868   0.764291  0.661891   
              None  10        0.701613  0.860684   0.784286  0.658920   
                    None      0.685591  0.857720   0.778868  0.631699   
 29.11.92.nii 0.25  10        0.876613  0.935398   0.878739  0.930936   
                    None      0.893721  0.928438   0.845406  0.968312   
              0.5   10        0.874635  0.935420   0.891310  0.935298   
                    None      0.893823  0.926261   0.830360  0.968986   
              2.0   10        0.881376  0.921811   0.789412  0.955082   
                    None      0.890187  0.924531   0.836116  0.970384   
              None  10        0.885036  0.927995   0.839876  0.947588   
                    None      0.875893  0.933206   0.889746  0.944720   
 29.6.67.nii  0.25  10        0.160242  0.224303   0.148000  1.000000   
                    None      0.292454  0.364492   0.185424  0.973283   
              0.5   10        0.205961  0.245232   0.157748  1.000000   
                    None      0.287469  0.344235   0.176671  0.938670   
              2.0   10        0.254523  0.310076   0.171193  0.998021   
                    None      0.266583  0.295124   0.171230  0.951802   
              None  10        0.284383  0.360144   0.179631  0.960666   
                    None      0.264229  0.284701   0.169730  0.951554   

                              Binary cross entropy  
Sample        Scale Rotation                        
 14.8.93.nii  0.25  10                    1.064528  
                    None                  1.002169  
              0.5   10                    1.098795  
                    None                  1.065558  
              2.0   10                    0.981847  
                    None                  1.046671  
              None  10                    1.011344  
                    None                  1.073150  
 29.11.92.nii 0.25  10                    0.101618  
                    None                  0.106072  
              0.5   10                    0.087141  
                    None                  0.121567  
              2.0   10                    0.190608  
                    None                  0.117907  
              None  10                    0.132598  
                    None                  0.086682  
 29.6.67.nii  0.25  10                   11.150045  
                    None                  8.127560  
              0.5   10                   10.139944  
                    None                  8.224020  
              2.0   10                    9.052817  
                    None                  8.702595  
              None  10                    8.298296  
                    None                  8.756887

In [4]:

df = pd.read_table('myapp.log',',|:|-|_|.h5',header=None  )  
df=df.drop(df.columns[[0,1,2,3,4,5,6,7,8,10,12,13,15,17,19,21,23]],axis=1)
df
df.columns=['Scale','Rotation', 'Sample', 'meanIOU','Accuracy', 'Precision','Recall','Binary cross entropy']
df
gk = df.groupby(['Sample'],axis=0).mean()
gk

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


,meanIOU,Accuracy,Precision,Recall,Binary cross entropy
Sample,,,,,
14.8.93.nii,0.702358,0.860122,0.776466,0.658863,1.043008
29.11.92.nii,0.883910,0.929132,0.850121,0.952663,0.118024
29.6.67.nii,0.251980,0.303538,0.169953,0.971750,9.056520
